## Async Version Agent Loop

In [5]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [1]:

# Toolkit

def calculator(expression: str) -> str:
    """Evaluates a single line of Python math expression. No imports or variables allowed.

    Args:
        expression (str): A mathematical expression using only numbers and basic operators (+,-,*,/,**,())

    Returns:
        The result of the calculation or an error message

    Examples:
        "2 + 2" -> "4"
        "3 * (17 + 4)" -> "63"
        "100 / 5" -> "20.0"
    """
    allowed = set("0123456789+-*/.() ")
    if not all(c in allowed for c in expression):
        return "Error: Invalid characters in expression"
    
    try:
        # eval is a dangerous function, use with caution
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}" 

def secret_digit(secret: str) -> str:
    return "The secret number is 7"

async def execute_tool(func, args) -> str:
    return func(**args)

In [2]:

# Parsers


import re
import json

def parse_thinking_from_response(response: str) -> str | None:
    """Parse a thinking from a response."""
    # re.DOTALL is used to all \n inside the think tags
    # ? matches lazily meaning we pick the content inside the first <think> </think>
    thinking = re.search(r'<think>(.*?)</think>', response, re.DOTALL)
    if thinking:
        return thinking.group(1)
    return None

def parse_tool_from_response(response: str) -> dict | None:
    """Parse a tool from a response."""
    tool_call = re.search(r'<tool>(.*?)</tool>', response, re.DOTALL)
    if tool_call:
        return json.loads(tool_call.group(1))
    return None


def parse_answer_from_response(response: str) -> str | None:
    """Parse an answer from a response."""
    answer = re.search(r'<answer>(.*?)</answer>', response, re.DOTALL)
    if answer:
        return answer.group(1)
    return None

In [3]:

# Async Version of Tool Call

async def call_tool(tool_call: dict) -> str:
    """Call a tool with the given tool call."""

    if tool_call['name'] == 'calculator':
        return calculator(tool_call['args']['calculator'])

    elif tool_call['name'] == 'secret_digit':
        return secret_digit(tool_call['args']['secret'])

    else:
        return f"Error: Tool {tool_call['name']} not found"

In [7]:
from openai import AsyncOpenAI
from typing import Union

oai = AsyncOpenAI()

async def agent_loop(question: str) -> Union[str, None]:

    system_prompt = """ 
    You are assistant who needs to find the secret number 
    You have access to the following tools:
    - calculator(calculator: str) -> str: Calculates the expressions. 
    - secret_digit(secret: str) -> str: Helps you find the secret number.

    You may call one tool per turn, for up to 10 turns, before giving your final answer.

    In each turn, you should respond in the following format:

    <think>
    [your thoughts here]
    </think>
    <tool>
    JSON with the following fields:
    - name: The name of the tool to call
    - args: A dictionary of arguments to pass to the tool (must be valid JSON)
    </tool>

    When you are done, give your final answer in the following format:

    <answer>
    [your final answer here]
    </answer>
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
    ]

    turns = 0
    
    while True:
        turns += 1

        try:
            response = await oai.chat.completions.create(
                model="gpt-4.1-nano",
                messages=messages, # type: ignore
            )

            response = response.choices[0].message.content # type: ignore

            # parse for thinking, tool call, and/or answer 
            maybe_thinking = parse_thinking_from_response(response) # type: ignore
            maybe_tool_call = parse_tool_from_response(response) # type: ignore
            maybe_answer = parse_answer_from_response(response) # type: ignore

        except Exception as e:
            print(f"Error: {e}")
            retries += 1
            
        print("=== Turn", turns, "===")

        if maybe_thinking: # type: ignore
            thinking = maybe_thinking.strip()
            print(f"Thinking: {thinking}")

        if maybe_tool_call: # type: ignore
            tool_call = maybe_tool_call
            tool_result = await call_tool(tool_call)
            print(f"Tool call: {tool_call}")
            print(f"Tool result: {tool_result[:100]}")
            messages.append({"role": "user", "content": tool_result})

        elif maybe_answer: # type: ignore
            final_answer = maybe_answer
            return final_answer

        else:
            raise ValueError



In [8]:
import asyncio
import nest_asyncio
nest_asyncio.apply() 

async def main():
    prompts = ["Could you tell me what the secret number is?", "What’s the hidden number, please?", "What’s the hidden number, please?", "Reveal the secret number, would you?",
               "Which number are you keeping under wraps?", "I’m curious—what’s the concealed number?"]

    agent_calls = [agent_loop(prompt) for prompt in prompts]
    results = await asyncio.gather(*agent_calls)
    return results


results = asyncio.run(main());
results

=== Turn 1 ===
Thinking: I need more information to identify the secret number. Are there any clues or hints provided? If not, perhaps I can try to gather some information by testing certain guesses or patterns.
Tool call: {'name': 'secret_digit', 'args': {'secret': ''}}
Tool result: The secret number is 7
=== Turn 1 ===
Thinking: I need more information to determine the secret number. Since I have access to a calculator tool, I could attempt some calculations to gather clues, but I don't have any initial data or hints. Perhaps I should try to get the secret digit directly by using the secret_digit tool.
Tool call: {'name': 'secret_digit', 'args': {'secret': ''}}
Tool result: The secret number is 7
=== Turn 1 ===
Thinking: I need more information or clues to determine the secret number. Since no specific hints or expressions are provided, I will attempt to probe the secret digit directly.
Tool call: {'name': 'secret_digit', 'args': {'secret': ''}}
Tool result: The secret number is 7
==

['\nThe secret number is 7.\n',
 '\n7\n',
 '\nThe secret number is 7.\n',
 '\n7\n',
 '\nThe secret number is 7.\n',
 '\nThe secret number is 7.\n']